In [2]:
import arxiv

In [8]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [11]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
tool=WikipediaQueryRun(api_wrapper=api_wrapper)
tool.name

'wikipedia'

In [13]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [17]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)


In [18]:
vectordb=FAISS.from_documents(documents,OllamaEmbeddings())


In [20]:
retriver=vectordb.as_retriever()


In [21]:
from langchain.tools.retriever import create_retriever_tool
retrival_tool=create_retriever_tool(retriver,"langsmith_search",
                      "Search for information about LangSmith, for any question on langSmith use this tool")




In [23]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
#Websites about research

In [24]:
arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [26]:
tools=[tool,arxiv,retrival_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/issac/project/envllm/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langsmith_search', description='Search for information about LangSmith, for any question on langSmith use this tool', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0xffff83f0dea0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_co

In [29]:
import os
import configparser
from langchain_community.llms import Ollama
configParser = configparser.RawConfigParser()   
configFilePath = r'/home/issac/project/llm/.env'
configParser.read(configFilePath)

os.environ["LANCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=configParser.get('config','lan_api')
os.environ["LANGCHAIN_PROJECT"]="lan_project 2"
llm=Ollama(model="llama2-uncensored")

In [1]:
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")

ImportError: Could not import langchainhub, please install with `pip install langchainhub`.

In [ ]:
## Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
